Scrape the neighbourhoods of Toronto talbe from Wikipedia using Beautiful Soup

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup


res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
findtables = soup.find_all('table')

# scrape all the tables found on the webpage
tables = pd.read_html(str(findtables))

# check the total number of tables found (4 in this case)
print(len(tables))

4


There are four tables on the page and the first one is the one we're looking for

In [4]:
# read the table into a dataframe, remove the space in the column header 'Postal Code' 

df = tables[0]
df.columns = ['PostalCode', 'Borough', 'Neighbourhood']

# check the shape of the dataframe (180 rows incl. header and 3 columns)
df.shape

(180, 3)

In [5]:
# filter out rows where borough has a valid value and store the filtered data into a new dataframe df_filtered 

df_filtered = df[df['Borough']!='Not assigned']

# check how many rows remain (103) and print out unique borough names to check if all 'not assigned' has been removed
print(df_filtered.shape)
print(df_filtered['Borough'].unique())

(103, 3)
['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Toronto/York'
 'Mississauga']


In [6]:
# add a column 'check_neighbourhood' that checks if any neighbourhood is 'Not assigned'
df_filtered['check_neighbourhood'] = df_filtered['Neighbourhood']!='Not assigned'
df_filtered

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighbourhood,check_neighbourhood
2,M3A,North York,Parkwoods,True
3,M4A,North York,Victoria Village,True
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",True
5,M6A,North York,"Lawrence Manor, Lawrence Heights",True
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",True
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",True
9,M1B,Scarborough,"Malvern, Rouge",True
11,M3B,North York,Don Mills,True
12,M4B,East York,"Parkview Hill, Woodbine Gardens",True
13,M5B,Downtown Toronto,"Garden District, Ryerson",True


In [7]:
# count the number of neighbours that have a valid value. The number equals row number indicating all neighbourhood is assigned a value
df_filtered['check_neighbourhood'].value_counts()

True    103
Name: check_neighbourhood, dtype: int64

In [8]:
# drop the checking column and store the cleaned data into df_clean

df_clean = df_filtered.drop(['check_neighbourhood'], axis=1)
df_clean.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df_clean.shape

(103, 3)

In [13]:
# Get latitudes and longtitudes of neighbourhoods

!pip install pgeocode
import pgeocode

pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df_clean['PostalCode'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        #print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

In [24]:
#Append latitudes and longtitudes to the dataframe (I made a typo of 'Longtitude but forgot to drop it')

import numpy as np
df_clean['Latitude'] = np.array(latitudes)
df_clean['Longitude'] = np.array(longitudes)
df_clean.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longtitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,-79.3889


In [35]:
# Filter out boroughs that only contain 'Toronto', save which into a dataframe df_toronto

df_toronto = df_clean[df_clean['Borough'].str.contains("Toronto")]
df_toronto = df_toronto.reset_index(drop = True)
df_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longtitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,-79.2941


In [17]:
# Import plotting libraries 
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim 

! pip install folium
import folium 

print("Libraries imported")

     |████████████████████████████████| 94 kB 4.9 MB/s  eta 0:00:01
Libraries imported


In [21]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [28]:
CLIENT_ID = '3ZMOVIUILUDPWA1SPXXYVSYGDPZBQAQPR3PJNPQQOMY1OSA2' 
CLIENT_SECRET = 'N3X4XJ4YGJS11MRNLYOF2QMGUDGDDBOQ3UTAR5EJQYUYYH12' 
VERSION = '20180605' 
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3ZMOVIUILUDPWA1SPXXYVSYGDPZBQAQPR3PJNPQQOMY1OSA2
CLIENT_SECRET:N3X4XJ4YGJS11MRNLYOF2QMGUDGDDBOQ3UTAR5EJQYUYYH12


Explore a neighbourhood in df_toronto that corresponds to a postcode/ borough on its own for simplicity. The first in the dataframe that meets this criterion is St. James Town

In [39]:
# Get the name, latitude and longitude of the St.James Town

neighborhood_latitude = df_toronto.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[3, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6513, -79.3756.


Get the top 100 venues that are in St. James Town within a radius of 500 meters.

In [41]:
# First create the GET request URL

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '604d708495476b2db23ad139'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'St. Lawrence',
  'headerFullLocation': 'St. Lawrence, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 91,
  'suggestedBounds': {'ne': {'lat': 43.6558000045, 'lng': -79.3693923061084},
   'sw': {'lat': 43.6467999955, 'lng': -79.38180769389162}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '583f54320923922e8141d360',
       'name': 'Crepe TO',
       'location': {'address': '52 Church Street',
        'lat': 43.650063279511016,
        'lng': -79.37458664523871,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65006327951101

Venue information is stored in the items key. First, use the get_category_type function from the Foursquare lab extract the category of the venue

In [42]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Import json_normalize to tranform JSON file into a pandas dataframe, and clean the json and structure it into a pandas dataframe.

In [43]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Crepe TO,Creperie,43.650063,-79.374587
1,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068
2,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
3,Hogtown Smoke,Food Truck,43.649287,-79.374689
4,Terroni,Italian Restaurant,43.650927,-79.375602


In [45]:
#Check the total number of venues returned

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

91 venues were returned by Foursquare.


Now create a function to repeat the same process to all the neighborhoods in Toronto

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called toronto_venues

In [55]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

In [56]:
# Check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(1533, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


Check how many venues were returned for each neighborhood

In [57]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,93,93,93,93,93,93
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,57,57,57,57,57,57
Christie,13,13,13,13,13,13
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


Check how many unique categories can be curated from all the returned venues

In [58]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


Analyse each neighbourhood

In [62]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Har

In [63]:
print(toronto_onehot.shape)

(1533, 215)


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [89]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.021505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.053763,0.000000,0.000000,0.000000,0.010753,0.000000,0.032258,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.021505,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.043011,0.000000,0.000000,0.021505,0.000000,0.000000,0.010753,0.032258,0.107527,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010753,0.000000,0.010753,0.00000,0.000000,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.021505,0.010753,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.000000,0.000000,0.00,0.000000,0.000000,0.021505,0.010753,0.000000,0.010753,0.000000,0.000000,0.000000,0.010753,0.010753,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,0.010753,0.010753,0.010753,0.000000,0.00,0.0

In [90]:
toronto_grouped.shape

(40, 215)

Print each neighborhood along with the top 5 most common venues

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Café,Hotel,Seafood Restaurant,Restaurant,Cocktail Bar,Japanese Restaurant,Beer Bar,Sandwich Place
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Breakfast Spot,Gift Shop,Thrift / Vintage Store,Italian Restaurant,Grocery Store,Gym,Hawaiian Restaurant,Falafel Restaurant
2,"Business reply mail Processing Centre, South C...",Restaurant,Italian Restaurant,Coffee Shop,Yoga Studio,Bookstore,Sushi Restaurant,Bank,Japanese Restaurant,Breakfast Spot,Deli / Bodega
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Bar,Café,Bakery,Gym / Fitness Center,Park,French Restaurant,Bank,Speakeasy
4,Central Bay Street,Coffee Shop,Sandwich Place,Bubble Tea Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Neighborhood,Seafood Restaurant,Breakfast Spot,Restaurant


Cluster Neighbourhoods

In [97]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [101]:
toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longtitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,-79.3626,2,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,-79.3889,2,Sushi Restaurant,Italian Restaurant,Park,Creperie,Café,Restaurant,Chinese Restaurant,Ramen Restaurant,Coffee Shop,College Cafeteria
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,-79.3783,2,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Movie Theater,Japanese Restaurant,Italian Restaurant,Café,Hotel,Tanning Salon
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,-79.3756,2,Café,Coffee Shop,Seafood Restaurant,Gastropub,Italian Restaurant,Cocktail Bar,Restaurant,Farmers Market,Hotel,Park
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,-79.2941,2,Pub,Neighborhood,Health Food Store,Gastropub,Bakery,Trail,Yoga Studio,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examin Clusters

In [104]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,-79.4205,0,Grocery Store,Café,Bank,Park,Candy Store,Athletics & Sports,Coffee Shop,Baby Store,Playground,Yoga Studio
10,Downtown Toronto,-79.3936,0,Harbor / Marina,Park,Café,Music Venue,Dog Run,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
24,Central Toronto,-79.4065,0,Gym Pool,Garden,Playground,Park,Grocery Store,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
30,Central Toronto,-79.3853,0,Park,Gym,Grocery Store,Thai Restaurant,Coworking Space,Creperie,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
34,Downtown Toronto,-79.3730,0,Park,Grocery Store,Playground,Candy Store,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [105]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,-79.4195,1,Home Service,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [106]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.3626,2,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,Downtown Toronto,-79.3889,2,Sushi Restaurant,Italian Restaurant,Park,Creperie,Café,Restaurant,Chinese Restaurant,Ramen Restaurant,Coffee Shop,College Cafeteria
2,Downtown Toronto,-79.3783,2,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Movie Theater,Japanese Restaurant,Italian Restaurant,Café,Hotel,Tanning Salon
3,Downtown Toronto,-79.3756,2,Café,Coffee Shop,Seafood Restaurant,Gastropub,Italian Restaurant,Cocktail Bar,Restaurant,Farmers Market,Hotel,Park
4,East Toronto,-79.2941,2,Pub,Neighborhood,Health Food Store,Gastropub,Bakery,Trail,Yoga Studio,Doner Restaurant,Donut Shop,Dumpling Restaurant
5,Downtown Toronto,-79.3754,2,Coffee Shop,Bakery,Café,Hotel,Seafood Restaurant,Restaurant,Cocktail Bar,Japanese Restaurant,Beer Bar,Sandwich Place
6,Downtown Toronto,-79.3860,2,Coffee Shop,Sandwich Place,Bubble Tea Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Neighborhood,Seafood Restaurant,Breakfast Spot,Restaurant
8,Downtown Toronto,-79.3833,2,Coffee Shop,Café,Gym,Hotel,Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Salad Place,Asian Restaurant
9,West Toronto,-79.4378,2,Grocery Store,Park,Bakery,Furniture / Home Store,Bar,Gym,Pet Store,Pharmacy,Pizza Place,Pool
11,West Toronto,-79.4177,2,Bar,Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Yoga Studio,Cuban Restaurant,Café,Record Shop,Pizza Place


In [107]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Toronto/York,-79.4839,3,Brewery,Athletics & Sports,Furniture / Home Store,Park,Grocery Store,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [108]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,-79.3935,4,Park,Photography Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
23,West Toronto,-79.4633,4,Park,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
